# Chapter 1.3

### Exercise 1.29

**Simpson’s Rule is a more accurate method of numerical integration than the method illustrated above. Using Simpson’s Rule, the integral of a function $f$ between $a$ and $b$ is approximated as**

$$
\frac{h}{3}(y_0 + 4y_1 + 2y_2 + 4y_3 + 2y_4 + \cdots + 2y_{n-2} + 4y_{n-1} + y_n),
$$

**where $h = (b - a)/n$, for some even integer $n$, and $y_k = f(a+kh)$. (Increasing $n$ increases the accuracy of the approximation.) Define a procedure that takes as arguments $f$ , $a$, $b$, and $n$ and returns the value of the integral, computed using Simpson’s Rule. Use your procedure to integrate cube between $0$ and $1$ (with $n = 100$ and $n = 1000$), and compare the results to those of the integral procedure shown above.**

In [1]:
(define (simpsons-rule f a b n)
  (define (y k h)
    (f (+ a (* k h))))
  (define (sum-iter k sum h)
    (if (> k n)
        sum
        (sum-iter (+ k 1) (+ sum (* (y k h)
                  (cond ((or (= k 0) (= k n)) 1)
                        ((even? k) 2)
                        (else 4)))) h)))
  (* (/ (- b a) (* 3 n)) (sum-iter 0 0 (/ (- b a) n))))
  

In [2]:
(define (cube x) (* x x x))
(simpsons-rule cube 0 1.0 100)

0.25000000000000006

In [3]:
(simpsons-rule cube 0 1.0 1000)

0.25000000000000006

As we see in the results above, output values produced by procedure implementing Simpson's rule is much more accurate than those produced by the procedure given in the text. Simpson's rules produces essentially the same value ($0.25000000000000006$) with $n=100$ and $n=1000$. This is practically equal (discounting the error produced by limited precision floating point arithmetic) to the true value $0.25$, whereas the given procedure produced $0.24999987500001$ with a step size of $0.001$, corresponding to $1000$ steps.

### Exercise 1.30

**The `sum` procedure above generates a linear recursion. The procedure can be rewritten so that the sum is performed iteratively. Show how to do this by filling in the missing expressions in the following definition:**

```scheme
(define (sum term a next b)
  (define (iter a result)
    (if ⟨??⟩
        ⟨??⟩
        (iter ⟨??⟩ ⟨??⟩)))
  (iter ⟨??⟩ ⟨??⟩))
```

The procedure given below performs the summation iteratively:

In [4]:
(define (sum term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (+ result (term a)))))
  (iter a 0))

In [5]:
(define (square x) (* x x))
(define (inc n) (+ n 1))
(sum square 1 inc 10)

385

In [6]:
(define (identity x) x)
(sum identity 1 inc 10)

55

### Exercise 1.31

**a. The `sum` procedure is only the simplest of a vast number of similar abstractions that can be captured as higher-order procedures. Write an analogous procedure called `product` that returns the product of the values of a function at points over a given range. Show how to define `factorial` in terms of `product`. Also use product to compute approximations to $\pi$ using the formula**

$$
\frac{\pi}{4} = \frac{2\cdot4\cdot4\cdot6\cdot6\cdot8\cdots}{3\cdot3\cdot5\cdot5\cdot7\cdot7\cdots}
$$

**b. If your product procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.**

a.

In [7]:
(define (product term a next b)
  (if (> a b)
      1
      (* (term a) (product term (next a) next b))))

(define (factorial n)
  (define (inc m) (+ m 1))
  (define (id m) m)
  (product id 1 inc n))
(factorial 5)

120

In [8]:
(define (pi-approx n)
  (define (term k)
    (/ (* 2 (+ (quotient k 2) 1)) (+ (* 2 (+ (quotient (- k 1) 2) 1)) 1)))
  (define (inc k) (+ k 1))
  (/ (* 4 (product term 1 inc n)) 1.0))    

In [9]:
(pi-approx 3)

2.8444444444444446

In [10]:
(pi-approx 10)

3.2751010413348074

In [11]:
(pi-approx 100)

3.1570301764551676

In [12]:
(pi-approx 10000)

3.1417497057380523

b. The `product` procedure given above generates a linear recursive process. The `iproduct` procedure given below generates an iterative process.

In [13]:
(define (iproduct term a next b)
  (define (product-iter t prod)
    (if (> t b)
        prod
        (product-iter (next t) (* (term t) prod))))
  (product-iter a 1))

In [14]:
(define (factorial n)
  (define (inc m) (+ m 1))
  (define (id m) m)
  (iproduct id 1 inc n))
(factorial 6)

720

### Exercise 1.32

**a. Show that sum and product (Exercise 1.31) are both special cases of a still more general notion called `accumulate` that combines a collection of terms, using some general accumulation function:**
```scheme
(accumulate combiner null-value term a next b)
```
**`accumulate` takes as arguments the same term and range specifications as `sum` and `product`, together with a `combiner` procedure (of two arguments) that specifies how the current term is to be combined with the accumulation of the preceding terms and a `null-value` that specifies what base value to use when the terms run out. Write `accumulate` and show how `sum` and `product` can both be defined as simple calls to `accumulate`.**

**b. If your accumulate procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.**

a. Given below is the `accumulate` procedure that works as a general accumulation function and `sum` and `product` implemented in terms of it.

In [15]:
(define (accumulate combiner null-value term a next b)
  (if (> a b)
      null-value
      (combiner (term a) (accumulate combiner null-value term (next a) next b))))

In [16]:
(define (sum term a next b)
  (accumulate + 0 term a next b))
(define (sum-natural-numbers n)
  (define (id m) m)
  (define (inc m) (+ m 1))
  (sum id 1 inc n))
(sum-natural-numbers 10)

55

In [17]:
(define (product term a next b)
  (accumulate * 1 term a next b))
(define (factorial n)
  (define (id m) m)
  (define (inc m) (+ m 1))
  (product id 1 inc n))
(factorial 6)

720

b. The `accumulate` procedure above generated a linear recursive process. The follwing `iaccumulate` procedure generates an iterative process. Then `iaccumulate` is used in the implementaion of `pi-leibniz`, a procedure that generates an approximation to $\pi$ using the *[Leibniz formula](https://en.wikipedia.org/wiki/Leibniz_formula_for_%CF%80)*.

In [18]:
(define (iaccumulate combiner null-value term a next b)
  (define (accumulate-iter n accumulation)
    (if (> n b)
        accumulation
        (accumulate-iter (next n) (combiner (term n) accumulation))))
  (accumulate-iter a null-value))

; Approximate $\pi$ using the Leibniz formula
(define (pi-leibniz n)
  (define (next m) (+ m 1))
  (define (term m)
    (if (odd? m) (/ 1.0 (- (* 2 m) 1)) (/ (- 1.0) (- (* 2 m) 1))))
  (* 4 (iaccumulate + 0 term 1 next n)))

In [19]:
(pi-leibniz 1000)

3.140592653839794

### Exercise 1.33

**You can obtain an even more general version of `accumulate` (Exercise 1.32) by introducing the notion of a *filter* on the terms to be combined. That is, combine only those terms derived from values in the range that satisfy a specified condition. The resulting `filtered-accumulate` abstraction takes the same arguments as `accumulate`, together with an additional predicate of one argument that specifies the filter. Write `filtered-accumulate` as a procedure. Show how to express the following using `filtered-accumulate`:**

**a. the sum of the squares of the prime numbers in the interval $a$ to $b$ (assuming that you have a `prime?` predicate already written)**

**b. the product of all the positive integers less than $n$ that are relatively prime to $n$ (i.e., all positive integers $i \lt n$ such that $\gcd(i,n)=1$**

In [20]:
(define (filtered-accumulate combiner null-value filter term a next b)
  (cond ((> a b) null-value)
        ((filter a) (combiner (term a) (filtered-accumulate combiner null-value filter term (next a) next b)))
        (else (filtered-accumulate combiner null-value filter term (next a) next b))))   

a. The procedure `sum-prime-squares` given below computes the sum of the squares of prime numbers in the interval $a$ to $b$.

In [21]:
(define (prime? n)
  (define (prime-iter m)
    (cond ((> (* m m) n) #t)
          ((= 0 (remainder n m)) #f)
          (else (prime-iter (+ m (if (= m 2) 1 2))))))
  (if (< n 2) #f (prime-iter 2)))

(define (sum-prime-squares a b)
  (define (inc m) (+ m 1))
  (define (square m) (* m m))
  (filtered-accumulate + 0 prime? square a inc b))

(sum-prime-squares 0 12)

208

b. The procedure `product-coprimes` given below computes the product of numbers below the given number which are relatively prime to it.

In [22]:
(define (gcd a b)
  (cond ((= b 0) a)
        ((> b a) (gcd b a))
        (else (gcd b (remainder a b)))))

(define (product-coprimes n)
  (define (inc m) (+ m 1))
  (define (id m) m)
  (define (coprime? m) (= (gcd n m) 1))
  (filtered-accumulate * 1 coprime? id 1 inc n))

(product-coprimes 15)

896896

### Exercise 1.34

**Suppose we define the procedure**
```scheme
(define (f g) (g 2))
```
**Then we have**
```scheme
(f square)
4
(f (lambda (z) (* z (+ z 1))))
6
```
**What happens if we (perversely) ask the interpreter to evaluate the combination `(f f)`? Explain.**

We have defined `f` as a procedure of a single parameter which evaluates to a value by applying the given parameter to the value 2. That is, it treats the argument (parameter) as a procedure and applies `2` to it. That works well when we pass the procedures like `square`, but when we try to pass `f` itself, what happens is the evaluator tries to evaluate `(f 2)`. To evaluate this expression, the evaluator needs to evaluate `(2 2)`, which is not a valid combination, because `2` is not a procedure. So, the interpreter throws an error.

### Exercise 1.35

**Show that the golden ratio $\varphi$ (Section 1.2.2) is a fixed point of the transformation $x \mapsto 1 + 1 / x$, and use this fact to compute $\varphi$ by means of the fixed-point procedure.**

In Section 1.2.2, $\varphi$ was defined as the value of $(\sqrt{5}+1)/2$. To see that $\varphi$ is a fixed point of the transformation $x \mapsto 1 + 1/x$, evaluate the right side with $x=\varphi$:
$$
1+\frac{1}{\varphi} = 1+\frac{1}{(\sqrt{5}+1)/2} = 1+\frac{2}{\sqrt{5}+1} = \frac{\sqrt{5}+1+2}{\sqrt{5}+1}\\ = \frac{(\sqrt{5}+3)(\sqrt{5}-1)}{(\sqrt{5}+1)(\sqrt{5}-1)} = \frac{\sqrt{5}^2-\sqrt{5}+3\sqrt{5}-3}{\sqrt{5}^2-1^2}\\ = \frac{(3-1)\sqrt{5}+(5-3)}{4}=\frac{2\sqrt{5}+2}{4}=\frac{\sqrt{5}+1}{2}=\varphi
$$

In [23]:
(define (fixed-point f first-guess)
  (define tolerance 0.00001)
  (define (close-enough? v1 v2)
    (< (abs (- v1 v2))
       tolerance))
  (define (try-value guess)
    (let ((next (f guess)))
      (if (close-enough? guess next)
          next
          (try-value next))))
  (try-value first-guess))
(fixed-point (lambda (x) (+ 1.0 (/ 1.0 x))) 1.0)

1.6180327868852458

### Exercise 1.36

**Modify `fixed-point` so that it prints the sequence of approximations it generates, using the `newline` and `display` primitives shown in Exercise 1.22. Then find a solution to $x^x = 1000$ by finding a fixed point of $x \mapsto \log(1000)/\log(x)$. (Use Scheme’s primitive `log` procedure, which computes natural logarithms.) Compare the number of steps this takes with and without average damping. (Note that you cannot start `fixed-point` with a guess of 1, as this would cause division by $\log(1) = 0$.)**

In [24]:
; We define log in python because callysto scheme does not implement it
(python-exec 
"
import math
def log(n):
 return math.log(n)
")

(define tolerance 0.00001)
(define (fixed-point-print f first-guess)
  (define (close-enough? v1 v2)
    (< (abs (- v1 v2))
       tolerance))
  (define (try-value guess)
    (display guess)
    (newline)
    (let ((next (f guess)))
      (if (close-enough? guess next)
          next
          (try-value next))))
  (try-value first-guess))

In [25]:
(fixed-point-print (lambda (x) (/ (log 1000) (log x))) 2.0)

2.0
9.965784284662087
3.004472209841214
6.279195757507157
3.759850702401539
5.215843784925895
4.182207192401397
4.8277650983445906
4.387593384662677
4.671250085763899
4.481403616895052
4.6053657460929
4.5230849678718865
4.577114682047341
4.541382480151454
4.564903245230833
4.549372679303342
4.559606491913287
4.552853875788271
4.557305529748263
4.554369064436181
4.556305311532999
4.555028263573554
4.555870396702851
4.555315001192079
4.5556812635433275
4.555439715736846
4.555599009998291
4.555493957531389
4.555563237292884
4.555517548417651
4.555547679306398
4.555527808516254
4.555540912917957


4.555532270803653

In the `fixed-point-print` procedure implemented above (without average damping), the process takes 34 steps to arrive at the solution to $x^x = 1000$, which is about $4.55532$.

In [26]:
(define (fixed-point-print-damp f first-guess)
  (define (close-enough? v1 v2)
    (< (abs (- v1 v2))
       tolerance))
  (define (average a b) (/ (+ a b) 2))
  (define (try-value guess)
    (display guess)
    (newline)
    (let ((next (average guess (f guess))))
      (if (close-enough? guess next)
          next
          (try-value next))))
  (try-value first-guess))

In [27]:
(fixed-point-print-damp (lambda (x) (/ (log 1000) (log x))) 2.0)

2.0
5.9828921423310435
4.922168721308343
4.628224318195455
4.568346513136242
4.5577305909237005
4.555909809045131
4.555599411610624
4.5555465521473675


4.555537551999825

In the `fixed-point-print-damp` procedure implemented with average damping, the process takes a mere 9 steps to arrive at an answer. This is significantly faster than not using average damping.

### Exercise 1.37

**a. An infinite *continued fraction* is an expression of the form**

$$
f = \cfrac{N_1}{D_1 + \cfrac{N_2}{D_2 + \cfrac{N_3}{D_3 + \ldots}}}
$$

**As an example, one can show that the infinite continued fraction expansion with the $N_i$ and the $D_i$ all equal to 1 produces $1/\varphi$, where $\varphi$ is the golden ratio (described in Section 1.2.2). One way to approximate an infinite continued fraction is to truncate the expansion after a given number of terms. Such a truncation - a so-called *k-term finite continued fraction* - has the form**

$$
\cfrac{N_1}{D_1 + \cfrac{N_2}{\ddots + \cfrac{N_k}{D_k}}}
$$

**Suppose that `n` and `d` are procedures of one argument (the term index $i$) that return the $N_i$ and $D_i$ of the terms of the continued fraction. Define a procedure `cont-frac` such that evaluating `(cont-frac n d k)` computes the value of the *k*-term finite continued fraction. Check your procedure by approximating $1/\varphi$ using**
```scheme
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           k)
```
**for successive values of `k`. How large must you make `k` in order to get an approximation that is accurate to 4 decimal places?**

**b. If your `cont-frac` procedure generates a recursive pro cess, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.**

a.

In [28]:
(define (cont-frac n d k)
  (define (cont-frac-compute i)
    (if (= k i)
        (/ (n i) (d i))
        (/ (n i) (+ (d i) (cont-frac-compute (+ i 1))))))
  (cont-frac-compute 1))

In [29]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           3)

0.6666666666666666

In [30]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           10)

0.6179775280898876

In [31]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           11)

0.6180555555555556

In [32]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           12)

0.6180257510729613

The value of $1/\varphi$ is $0.6180339887\ldots$. To get a value that is accurate to 4 decimal places, we must make k as large as 12.

b. The `cont-frac` procedure given above generated a recursive process. Given below is an implementation of `cont-frac` that generates an iterative process:

In [33]:
(define (cont-frac n d k)
  (define (cont-frac-iter acc i)
    (if (= i 0)
        acc
        (cont-frac-iter (/ (n i) (+ (d i) acc)) (- i 1))))
  (cont-frac-iter 0 k))

In [34]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           12)

0.6180257510729613

### Exercise 1.38

**In 1737, the Swiss mathematician Leonhard Euler published a memoir *De Fractionibus Continuis*, which included a continued fraction expansion for $e − 2$, where $e$ is the base of the natural logarithms. In this fraction, the $N_i$ are all 1, and the $D_i$ are successively 1, 2, 1, 1, 4, 1, 1, 6, 1, 1, 8, .... Write a program that uses your `cont-frac` procedure from Exercise 1.37 to approximate $e$, based on Euler’s expansion.**

In [35]:
(define (e-frac-denominator i)
  (if (= (remainder (- i 1) 3) 1)
      (* 2 (+ 1 (quotient (- i 1) 3)))
      1))
(define (approx-e k)
  (+ 2 (cont-frac (lambda (i) 1.0)
                  e-frac-denominator
                  k)))

In [36]:
(approx-e 10)

2.7182817182817183

### Exercise 1.39

**A continued fraction representation of the tangent function was published in 1770 by the German mathematician J.H. Lambert:**
$$
\tan{x} = \cfrac{x}{1-\cfrac{x^2}{3-\cfrac{x^2}{5-\ldots}}}
$$
**where $x$ is in radians. Define a procedure `(tan-cf x k)` that computes an approximation to the tangent function based on Lambert’s formula. `k` specifies the number of terms to compute, as in Exercise 1.37.**

In [37]:
(define (tan-cf x k)
  (define (n i)
    (if (= i 1) x (- (* x x))))
  (define (d i) (- (* 2 i) 1))
  (cont-frac n d k))

In [38]:
(tan-cf 0.785398 15)

0.9999996732051568

Here, the argument `x` is approximately equal to $\pi/4$, and the result of the procedure, as expected, is approximately equal to 1, which is the value of $\tan{(\pi/4)}$

### Exercise 1.40

**Define a procedure `cubic` that can be used together with the `newtons-method` procedure in expressions of the form**
```scheme
(newtons-method (cubic a b c) 1)
```
**to approximate zeros of the cubic $x^3 + ax^2 + bx + c$.**

In [39]:
(define (cubic a b c)
  (lambda (x) (+ (* x x x) (* a x x) (* b x) c)))

### Exercise 1.41

**Define a procedure `double` that takes a procedure of one argument as argument and returns a procedure that applies the original procedure twice. For example, if `inc` is a procedure that adds 1 to its argument, then `(double inc)` should be a procedure that adds 2. What value is returned by**
```scheme
(((double (double double)) inc) 5)
```

In [40]:
(define (double f)
  (lambda (x) (f (f x))))

In [41]:
(define (inc n) (+ n 1))
(((double (double double)) inc) 5)

21

The value retured is 21 because `(((double (double double)) inc) 5)` evaluates as is shown below:
```scheme
(((double (double double)) inc) 5)
(((double (lambda (x) (double (double x)))) inc) 5)
(((lambda (x) (double (double (double (double x))))) inc) 5)
((double (double (double (double inc)))) 5)
```

In other words, `double` is applied to `inc` and again to the result and so on, 4 times in total. In the end, the resulting procedure adds 16 $(2^4)$ to its argument. The expression evaluates to $5 + 16 = 21$.

### Exercise 1.42

**Let $f$ and $g$ be two one-argument functions. The *composition* $f$ after $g$ is defined to be the function $x \mapsto f(g(x))$. Define a procedure `compose` that implements composition. For example, if `inc` is a procedure that adds 1 to its argument,**
```scheme
((compose square inc) 6)
49
```

In [42]:
(define (compose f g)
  (lambda (x) (f (g x))))

The above definition of `compose` implements composition, as is verified by the following evaluation:

In [43]:
(define (square x) (* x x))
(define (inc x) (+ 1 x))
((compose square inc) 6)

49

### Exercise 1.43

**If $f$ is a numerical function and $n$ is a positive integer, then we can form the $n_{th}$ repeated application of $f$ , which is defined to be the function whose value at $x$ is $f(f(\ldots(f(x))\ldots))$. For example, if $f$ is the function $ x \mapsto x + 1$, then the $n_{th}$ repeated application of $f$ is the function $x \mapsto x + n$. If $f$ is the operation of squaring a number, then the $n_{th}$ repeated application of $f$ is the function that raises its argument to the $2^n-th$ power. Write a procedure that takes as inputs a procedure that computes $f$ and a positive integer $n$ and returns the procedure that computes the $n_{th}$ repeated application of $f$ . Your procedure should be able to be used as follows:**
```scheme
((repeated square 2) 5)                                                            
625
```
**Hint: You may find it convenient to use compose from Exercise 1.42.**

In [44]:
(define (compose f g)
  (lambda (x) (f (g x))))
(define (square n) (* n n))

(define (repeated f n)
  (if (< n 2) f (compose f (repeated f (- n 1)))))

((repeated square 2) 5)

625

### Exercise 1.44

**The idea of *smoothing* a function is an important concept in signal processing. If $f$ is a function and $dx$ is some small number, then the smoothed version of $f$ is the function whose value at a point $x$ is the average of $f(x-dx)$, $f(x)$, and $f(x+dx)$. Write a procedure `smooth` that takes as input a procedure that computes $f$ and returns a procedure that computes the smoothed $f$. It is sometimes valuable to repeatedly smooth a function (that is, smooth the smoothed function, and so on) to obtain the *n-fold smoothed function*. Show how to generate the n-fold smoothed function of any given function using `smooth` and `repeated` from Exercise 1.43.**                                                                                                                                                                

In [45]:
(define dx 0.0001)
(define (smooth f)
  (lambda (x) (/ (+ (f (+ x dx)) (f x) (f (- x dx))) 3.0)))

(define (n-fold-smooth f n)
  ((repeated smooth n) f))

((n-fold-smooth square 5) 2)

4.000000033333335

### Exercise 1.45

**We saw in Section 1.3.3 that attempting to compute square roots by naively finding a fixed point of $y \mapsto x/y$ does not converge, and that this can be fixed by average damping. The same method works for finding cube roots as fixed points of the average-damped $y \mapsto x/y^2$. Unfortunately, the process does not work for fourth roots - single average damp is not enough to make a fixed-point search for $y \mapsto x/y^3$ converge. On the other hand, if we average damp twice (i.e., use the average damp of the average damp of $y \mapsto x/y^3$) the fixed-point search does converge. Do some experiments to determine how many average damps are required to compute $n^{th}$  roots as a fixed point search based upon repeated average damping of $ x \mapsto x/y^{n−1}$. Use this to implement a simple procedure for computing $n^{th}$  roots using `fixed-point`, `average-damp`, and the `repeated` procedure of Exercise 1.43. Assume that any arithmetic operations you need are available as primitives.**

To enable experimentation, we define a new `fixed-point` procedure that gives up trying to find the fixed point if our guess doesn't converge after a certain number of steps.

In [46]:
(define (fixed-point-bounded f guess)
  (define tolerance 0.00001)
  (define (close-enough? a b) (< (abs (- a b)) tolerance))
  (define (declare-no-convergence return-value)
    (display "The function has failed to converge!")
    (newline)
    return-value)
  (define (fixed-point-iter guess max-steps)
    (let ((next (f guess)))
      (cond ((close-enough? guess next) next)
            ((> max-steps 0) (fixed-point-iter next (- max-steps 1)))
            (else (declare-no-convergence "?")))))
  (fixed-point-iter guess 5000))

In [47]:
(define (average x y) (/ (+ x y) 2.0))
(define (average-damp f) (lambda (x) (average x (f x))))
; Compute the n-th root of x, by finding the fixed point of y |--> x/y^(n-1)
; and using d-fold average damping.
(define (nth-root-damped d n x)
  (define (g y) (/ x (expt y (- n 1))))
  (define (d-average-damp f) ((repeated average-damp d) f))
  (fixed-point-bounded (d-average-damp g) 1.0))

In [48]:
(nth-root-damped 1 2 9)

3.0

In [49]:
(nth-root-damped 1 4 625)

The function has failed to converge!


"?"

In [50]:
(nth-root-damped 2 4 625)

5.000000000004688

In [51]:
(nth-root-damped 2 6 15625)

5.000003268647355

In [52]:
(nth-root-damped 2 7 78125)

5.000004267274231

In [53]:
(nth-root-damped 2 8 390625)

The function has failed to converge!


"?"

In [54]:
(nth-root-damped 3 8 390625)

5.000000000000416

In [55]:
(nth-root-damped 3 16 390625)

The function has failed to converge!


"?"

In [56]:
(nth-root-damped 4 16 65536)

2.000000000076957

The experiments seem to be showing a pattern - that the number of `average-damp`s applied should be at least the $\log_2(n)$ (logarithm base 2 of n) when we need to calculate the nth root. Below we implement a n-th root procedure using this observation.

In [57]:
; Because apparently, callysto scheme doesn't have the floor function
(python-exec "from math import floor")

(define (nth-root n x)
  (define (g y) (/ x (expt y (- n 1))))
  (define (damped f) ((repeated average-damp (floor (/ (log n) (log 2)))) f))
  (fixed-point (damped g) 1.0))

In [58]:
(nth-root 10 1024)

2.000001183010332

In [59]:
(nth-root 4 625)

5.000000000004688

### Exercise 1.46

**Several of the numerical methods described in this chapter are instances of an extremely general computational strategy known as *iterative improvement*. Iterative improvement says that, to compute something, we start with an initial guess for the answer, test if the guess is good enough, and otherwise improve the guess and continue the process using the improved guess as the new guess. Write a procedure `iterative-improve` that takes two procedures as arguments: a method for telling whether a guess is good enough and a method for improving a guess. `iterative-improve` should return as its value a procedure that takes a guess as argument and keeps improving the guess until it is good enough. Rewrite the `sqrt` procedure of Section 1.1.7 and the `fixed-point` procedure of Section 1.3.3 in terms of `iterative-improve`.**

In [60]:
(define (iterative-improve improve good-enough?)
  (define (iterate guess)
    (if (good-enough? guess)
        (improve guess)
        (iterate (improve guess))))
  iterate)

(define (sqrt x)
  (define (improve guess)
    (average guess (/ x guess)))
  (define (good-enough? guess)
    (< (abs (- (* guess guess) x)) 0.00001))
  ((iterative-improve improve good-enough?) 1.0))

(define (fixed-point f)
  (define (improve guess)
    (average guess (f guess)))
  (define (good-enough? guess)
    (< (abs (- guess (f guess))) 0.00001))
  ((iterative-improve improve good-enough?) 1.0))

In [61]:
(sqrt 256)

16.0

In [62]:
(fixed-point (lambda (x) (/ 27.0 (* x x))))

3.000001383950946